# Phase 6: Fairness Evaluation & Mitigation

We’ll:
1. Compute fairness metrics with AIF360 & Fairlearn  
2. Apply reweighting to mitigate bias  
3. Re-train and compare metrics  


In [1]:
import sys, os
sys.path.insert(0, os.path.abspath(".."))

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from src.preprocessing import load_data, split_and_preprocess
from src.fairness import evaluate_fairness, run_reweighing


pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[inFairness]'


In [2]:
# 1) Load the raw DataFrame
df = load_data()

# 2) Split raw DataFrame for fairness evaluation
df_train_raw, df_test_raw = train_test_split(
    df,
    test_size=0.2,
    stratify=df["Risk"],
    random_state=42
)

# 3) Preprocess and split for modeling
X_train, X_test, y_train, y_test, preprocessor = split_and_preprocess(df)


In [3]:
# Protected attribute and privileged codes
prot_attr = "Personal_Status_Sex"
priv_cats = ["A91", "A93", "A94"]   # male codes

# Train baseline RandomForest
clf_base = RandomForestClassifier(n_estimators=100, random_state=42)
clf_base.fit(X_train, y_train)

# Evaluate fairness on the held-out raw test set
aif_base, fl_base = evaluate_fairness(
    X_test,            # preprocessed features
    y_test,            # numeric labels (1=good, 0=bad)
    preprocessor,
    clf_base,
    df_test_raw,       # raw DataFrame (with string codes)
    prot_attr=prot_attr,
    priv_categories=priv_cats
)

print("Baseline AIF360 metrics:", aif_base)
print("Baseline Fairlearn metrics by group:\n", fl_base)


Baseline AIF360 metrics: {'stat_parity_diff': np.float64(-0.00952380952380949), 'eq_opp_diff': np.float64(0.030000000000000027)}
Baseline Fairlearn metrics by group:
 {'selection_rate': {0: 0.7833333333333333, 1: 0.7928571428571428}, 'true_positive_rate': {0: 0.9, 1: 0.87}}


In [10]:
# 1) Get the AIF360 StandardDataset back
dataset_rw = run_reweighing(
    df_train_raw,
    priv_attr=prot_attr,        # note the correct keyword
    priv_categories=priv_cats
)

# 2) Extract the weight array
w_train = dataset_rw.instance_weights

# 3) Preprocess the raw training features
X_rw = preprocessor.fit_transform(df_train_raw.drop("Risk", axis=1))
y_rw = df_train_raw["Risk"].map({"good": 1, "bad": 0}).values

# 4) Retrain classifier with those sample weights
clf_rw = RandomForestClassifier(n_estimators=100, random_state=42)
clf_rw.fit(X_rw, y_rw, sample_weight=w_train)

# 5) Evaluate fairness on the held‐out test split
aif_rw, fl_rw = evaluate_fairness(
    X_test,
    y_test,
    X_rw,
    clf_rw,
    df_test_raw,
    prot_attr=prot_attr,
    priv_categories=priv_cats
)

print("Post‐Reweighing AIF360 metrics:", aif_rw)
print("Post‐Reweighing Fairlearn metrics by group:\n", fl_rw)


Post‐Reweighing AIF360 metrics: {'stat_parity_diff': np.float64(-0.030952380952380953), 'eq_opp_diff': np.float64(-0.03500000000000003)}
Post‐Reweighing Fairlearn metrics by group:
 {'selection_rate': {0: 0.7833333333333333, 1: 0.8142857142857143}, 'true_positive_rate': {0: 0.875, 1: 0.91}}
